In [1]:
# 0. Importing the libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

# 1. Load the dataset
df = pd.read_csv('sheep_data.csv')
df.shape

(64626, 83)

In [2]:
# 2. Normalise the features

# Separate features and labels
X = df.iloc[:, :-1]  # All columns except the last are features
y = df.iloc[:, -1]   # The last column is the label

# Normalize the features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


In [3]:
# 3. Encode the categorical labels

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
label_names = encoder.classes_
label_names, y_encoded

(array(['grazing', 'resting', 'scratching', 'standing', 'walking'],
       dtype=object),
 array([0, 0, 0, ..., 4, 4, 4]))

In [4]:
# 4. Reshape data for CNN input (Batch, Channels, Length)
X_reshaped = X_normalized.reshape(-1,1,82)
X_reshaped.shape

(64626, 1, 82)

In [5]:
# 5. Convert to PyTorch tensors
X_tensor = torch.tensor(X_reshaped, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

In [6]:
X_tensor.shape, y_tensor.shape

(torch.Size([64626, 1, 82]), torch.Size([64626]))

In [7]:
# 6. Create Dataset and DataLoader

dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training and temporary dataset
train_dataset, temp_dataset = train_test_split(dataset, test_size=0.3, 
                                               random_state=42)

# Split the temporary dataset into validation and test datasets
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=0.5, 
                                             random_state=42)

# Create DataLoaders for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


In [8]:
# 7. Define the CNN model

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class AccelerometerCNN(nn.Module):
    def __init__(self):
        super(AccelerometerCNN, self).__init__()
        # Assuming input shape [batch_size, 1, 82]
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1) # Output: [batch_size, 32, 82]
        self.pool = nn.MaxPool1d(2) # Output: [batch_size, 32, 41]
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1) # Output: [batch_size, 64, 41]
        # Apply pooling again: [batch_size, 64, 20]
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64 * 20, 128) # Fully connected layers
        self.fc2 = nn.Linear(128, 5) # Output layer, 5 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 20) # Flatten for fully connected layer
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [9]:
# 8. Setup Early Stopping

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [10]:
# 9. Initialize the Model, Loss Function, Optimizer, and Move to GPU

# Initialize model
model = AccelerometerCNN()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function with L2 Regularization (Weight Decay)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # L2 regularization

# Early stopping
early_stopping = EarlyStopping(patience=20, min_delta=0.01)


In [11]:
# 10. Model training and evaluation on validation set

from sklearn.metrics import classification_report, accuracy_score

max_epochs = 1000

for epoch in range(max_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    train_accuracy = 100 * correct / total

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
    
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break


Epoch 1, Train Loss: 0.3322, Train Accuracy: 85.71%, Val Loss: 0.1897, Val Accuracy: 92.01%
Epoch 2, Train Loss: 0.1939, Train Accuracy: 92.02%, Val Loss: 0.1500, Val Accuracy: 93.51%
Epoch 3, Train Loss: 0.1601, Train Accuracy: 93.43%, Val Loss: 0.1311, Val Accuracy: 94.66%
Epoch 4, Train Loss: 0.1400, Train Accuracy: 94.38%, Val Loss: 0.1143, Val Accuracy: 95.29%
Epoch 5, Train Loss: 0.1229, Train Accuracy: 95.06%, Val Loss: 0.1133, Val Accuracy: 95.37%
Epoch 6, Train Loss: 0.1105, Train Accuracy: 95.46%, Val Loss: 0.0971, Val Accuracy: 96.06%
Epoch 7, Train Loss: 0.1026, Train Accuracy: 95.87%, Val Loss: 0.0913, Val Accuracy: 96.28%
Epoch 8, Train Loss: 0.0934, Train Accuracy: 96.24%, Val Loss: 0.0908, Val Accuracy: 96.20%
Epoch 9, Train Loss: 0.0864, Train Accuracy: 96.57%, Val Loss: 0.0916, Val Accuracy: 96.36%
Epoch 10, Train Loss: 0.0817, Train Accuracy: 96.72%, Val Loss: 0.0822, Val Accuracy: 96.63%
Epoch 11, Train Loss: 0.0750, Train Accuracy: 96.99%, Val Loss: 0.0808, Val Acc

In [12]:
# 11. Evaluate the model
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        y_true.extend(target.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Classification report with names of the labels
target_names = encoder.classes_
classification_report = classification_report(y_true, y_pred, target_names=target_names)
print("Classification Report:\n", classification_report)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


Classification Report:
               precision    recall  f1-score   support

     grazing       0.99      1.00      1.00      2416
     resting       0.98      0.97      0.98      4166
  scratching       0.96      0.89      0.92        88
    standing       0.95      0.96      0.96      2058
     walking       1.00      1.00      1.00       966

    accuracy                           0.98      9694
   macro avg       0.98      0.96      0.97      9694
weighted avg       0.98      0.98      0.98      9694

Accuracy: 0.9789560552919332
